In [2]:
import pandas as pd 

In [3]:
import os

In [7]:
print(os.listdir("../my-flask-app"))

['.ipynb_checkpoints', 'aldo.ipynb', 'algo.py', 'app.py', 'ForumServer.py', 'friends.py', 'loginandsignup.py', 'logintest.py', 'package.json', 'postcreation.py', 'poststesting.py', 'tempPython.py', 'uploaddatabase.py', 'uploads', 'uploadstory.py', 'venv', '__pycache__']


In [8]:
talent = pd.read_csv("talent.csv")

In [10]:
talent.columns

Index(['id', 'name', 'price', 'reactions', 'stars', 'joined', 'categories',
       'in_13_reasons_why', 'in_90_day_fiance', 'in_actors',
       ...
       'in_ufc', 'in_vanderpump_rules', 'in_venture_capitalists', 'in_viners',
       'in_vlog_squad', 'in_voice_actors', 'in_winter_sports', 'in_writers',
       'in_younow', 'in_youtubers'],
      dtype='object', length=110)

In [12]:
print(talent.head())


               id             name  price  reactions  stars         joined  \
0     perezhilton     Perez Hilton   27.0        924    5.0     April 2018   
1        andydick        Andy Dick   99.0        340    4.9   October 2018   
2         tjlavin         TJ Lavin   80.0        291    5.0  February 2018   
3  carsonkressley  Carson Kressley   59.0        290    5.0   October 2018   
4        riffraff        RiFF RAFF   75.0        402    4.7  December 2017   

                                          categories  in_13_reasons_why  \
0         ['Reality TV', 'Commentators', 'Featured']                  0   
1  ['Reality TV', 'Comedians', 'Featured', 'Actors']                  0   
2  ['Reality TV', 'Riders', 'Featured', 'Extreme ...                  0   
3  ['Reality TV', 'Bravo', 'Stylists', 'Featured'...                  0   
4               ['Rappers', 'Featured', 'Musicians']                  0   

   in_90_day_fiance  in_actors  ...  in_ufc  in_vanderpump_rules  \
0           

In [13]:
print(talent.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5392 entries, 0 to 5391
Columns: 110 entries, id to in_youtubers
dtypes: float64(2), int64(104), object(4)
memory usage: 4.5+ MB
None


In [15]:
da_columns = [col for col in talent.columns if col.startswith("in_")]

In [16]:
da_columns

['in_13_reasons_why',
 'in_90_day_fiance',
 'in_actors',
 'in_adult_film_stars',
 'in_american_idol',
 'in_animals',
 'in_artists',
 'in_athletes',
 'in_authors',
 'in_baseball',
 'in_basketball',
 'in_below_deck',
 'in_bloggers',
 'in_bodybuilders',
 'in_boxing',
 'in_bravo',
 'in_british_stars',
 'in_broadway',
 'in_browse_featured',
 'in_cameo_la',
 'in_cameo_team',
 'in_chefs',
 'in_comedians',
 'in_commentators',
 'in_country_music',
 'in_cricket',
 'in_djs',
 'in_dancers',
 'in_disney',
 'in_drag_queens',
 'in_entrepreneurs',
 'in_extreme_sports',
 'in_family',
 'in_fashion',
 'in_featured',
 'in_fighters',
 'in_fitness',
 'in_food',
 'in_football',
 'in_for_charity',
 'in_game_of_thrones',
 'in_gamers',
 'in_golf',
 'in_hall_of_famers',
 'in_harry_potter',
 'in_hockey',
 'in_instagram_influencers',
 'in_legends',
 'in_lifestyle',
 'in_mtv',
 'in_magicians',
 'in_models',
 'in_motivational_speakers',
 'in_musicians',
 'in_nfl_draft_2019',
 'in_new',
 'in_olympics',
 'in_one_tree_